In [1]:
from datetime import timedelta
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from itertools import chain, combinations, permutations
import numpy as np

In [2]:
df = pd.read_parquet('../../Sonar/seatunnel_all_information.parquet')

In [3]:
df['time'] = df['total_time'].astype('int64')

In [4]:
X = df.loc[:, ~df.columns.isin(['begin_sha', 'end_sha', 'begin_time', 'end_time',
                                'total_time',
                                'time'
                                ])]

In [5]:
X.head(1)

,commits,additions,deletions,changed_files,begin_Dispensables,begin_Bloaters,begin_Change Preventers,begin_Couplers,begin_Object-Orientation Abusers,end_Dispensables,end_Bloaters,end_Change Preventers,end_Couplers,end_Object-Orientation Abusers,time
0,1,10,4,3,500.0,262.0,262.0,304.0,304.0,500.0,261.0,261.0,304.0,693.0,264468000000000


In [6]:
# Step 1: Loop through combinations
def column_combinations(data_frame):
    columns = data_frame.columns
    col_combinations = []

    for r in range(1, len(columns) + 1):
        for combo in combinations(columns, r):
            col_combinations.append(combo)

    return col_combinations

# Example usage:
data = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
combinations = column_combinations(X)
print(combinations)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
# # Step 2: Prepare the data set from loop combinations
# def prepare_data_set(data_frame, combination):
#     subset = data_frame[list(combination)]
#     return subset

In [8]:
# Step 3: Use MinMaxScaler
def scale_data(data,columns):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data[columns])

    return pd.DataFrame(scaled_data)

In [9]:
# co = scale_data(X, X.columns)
# co

In [10]:
# Step 4: Calculate silhouette score and sort clusters
def get_optimal_clusters(data):
    scores = []
    for n_clusters in range(2, len(data)):
        km = KMeans(n_clusters = n_clusters)
        km.fit(data)
        sil_avg = silhouette_score(data, km.labels_).round(4)
        scores.append([sil_avg , n_clusters])
    # scores.sort(reverse=True)
    # return scores[:5]
    return scores

In [11]:
# # Step 5: Perform KMeans clustering
# def perform_clustering(data_original, df_process,  num_clusters):
#     kmeans = KMeans(n_clusters=num_clusters)
#     # kmeans.fit_predict(df_process)
#     cluster_labels = kmeans.fit_predict(df_process)
#     data_original['cluster'] = cluster_labels
#     return data_original

In [12]:
# data = X.iloc[:, [0,1,2]]

In [13]:
# num_clusters = get_optimal_clusters(data)

In [14]:
# num_clusters

In [15]:
# num_clusters.sort()

In [16]:
# num_clusters = num_clusters[::-1]

In [17]:
# num_clusters[:5]

In [18]:
# from sklearn.cluster import KMeans

# kmeans = KMeans(n_clusters=3)
# kmeans

In [19]:
# cluster_labels = kmeans.fit_predict(X)

In [20]:
# cluster_labels

In [21]:
# df['cluster'] = cluster_labels

In [22]:
# clustered_data = perform_clustering(df, X, num_clusters)

In [23]:
# Step 6, 7, and 8: Get cluster information
def get_cluster_info(cluster_data):
    cluster_info = []
    # Replace with your time column name
    time_column = 'total_time'

    for cluster_num in cluster_data['cluster'].unique():
        cluster_subset = cluster_data[cluster_data['cluster'] == cluster_num]
        time_min = cluster_subset[time_column].min()
        time_max = cluster_subset[time_column].max()
        time_std = cluster_subset[time_column].std()
        time_minus_2std = time_min - 2 * time_std
        time_minus_1std = time_min - time_std
        time_plus_1std = time_max + time_std
        time_plus_2std = time_max + 2 * time_std
        time_avg = cluster_subset[(cluster_subset[time_column] >= time_minus_1std) & (cluster_subset[time_column] <= time_plus_1std)][time_column].mean()

        cluster_info.append({
            'cluster_num': cluster_num,
            'time_min': time_min,
            'time_max': time_max,
            'time_minus_2std': time_minus_2std,
            'time_minus_1std': time_minus_1std,
            'time_avg': time_avg,
            'time_plus_1std': time_plus_1std,
            'time_plus_2std': time_plus_2std
        })

    return cluster_info

In [ ]:
from itertools import combinations
import pandas as pd

# Step 1: Loop through combinations
def column_combinations(data_frame):
    columns = data_frame.columns
    col_combinations = []

    for r in range(1, len(columns) + 1):
        for combo in combinations(columns, r):
            col_combinations.append(combo)

    return col_combinations

# Main function
# def main():
    data = X
    combinations_list = column_combinations(data)
    print('Combinations_list ::::' , combinations_list)

    results = []
    for combination in combinations_list:
        # print('DATA ::', data)
        # print('=====================')

        scaled_data = scale_data(data , data.columns)
        # print('SCALED ::', scaled_data)
        # print('====================')

        num_clusters = get_optimal_clusters(scaled_data)
        # print('NUM ::', num_clusters)
        # print('====================')
        #
        num_clusters.sort()
        num_clusters = num_clusters[::-1]
        num_clusters = num_clusters[:3]

        kmeans = KMeans(n_clusters=3)
        cluster_labels = kmeans.fit_predict(X)
        df['cluster'] = cluster_labels

        clustered_data = df
        # print("CLUSTERRED DATA ::", clustered_data)
        # print('====================')

        cluster_info = get_cluster_info(clustered_data)
        # print("CL cluster_info ::" , cluster_info)
        # print('====================')


        for info in cluster_info:
            result_dict = {
                'combination': combination,
                'cluster_best': max(num_clusters[0]),
                'cluster_top_3' : num_clusters,
                # 'cluster_num': info['cluster_num'],
                # 'silhouette_score': get_optimal_clusters(scaled_data),
                # 'cluster_sort': num_clusters,
                'time_min': info['time_min'],
                'time_max': info['time_max'],
                'time_minus_2': info['time_minus_2std'],
                'time_minus_1': info['time_minus_1std'],
                'time_avg': info['time_avg'],
                'time_plus_1': info['time_plus_1std'],
                'time_plus_2': info['time_plus_2std']
            }
            results.append(result_dict)

    results_df = pd.DataFrame(results)
    results_df.to_csv('output_seatunnel.csv', index=False)  # Save results to CSV

# if __name__ == "__main__":
#     main()


C:\Users\patcharaprapa_k\AppData\Local\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\patcharaprapa_k\AppData\Local\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=5.
  warnings.warn(
C:\Users\patcharaprapa_k\AppData\Local\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\patcharaprapa_k\AppData\Local\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with M

In [ ]:
X.columns[:3]

In [ ]:
X.iloc[:, [0,1,2,14]]

In [ ]:
rf = pd.read_csv('output_results_2.csv')
rf

In [ ]:
rf['time_min'] = pd.to_timedelta(rf['time_min'])
rf['time_max'] = pd.to_timedelta(rf['time_max'])
rf['time_minus_1std'] = pd.to_timedelta(rf['time_minus_1std'])
rf['time_minus_2std'] = pd.to_timedelta(rf['time_minus_2std'])
rf['time_plus_1std'] = pd.to_timedelta(rf['time_plus_1std'])
rf['time_plus_2std'] = pd.to_timedelta(rf['time_plus_2std'])

In [ ]:
rf